# Data Vis: Plotting Time Series Data
* Notebook 1: Smoothing and Trends

## Setup

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
import seaborn as sns
from IPython.display import HTML


# Data

In this notebook, we will use a private dataset about (solar) power generation and use of a single family house in Germany. The dataset contains the following columns:
- `timestamp`: The date and time of the measurement. The data is recorded every 1 hour.
- `total_consumption_kw`: The amount of power consumed per hour in kilowatts.
- `from_grid_kw`: The amount of power provided from the grid per hour in kilowatts.
- `from_pv_kw`: The amount of power generated by the solar panels per hour in kilowatts.
- `from_battery_kw`: The amount of power provided by the battery per hour in kilowatts.
- `to_grid_kw`: The amount of power provided to the grid per hour in kilowatts.
- `to_battery_kw`: The amount of power provided to the battery per hour in kilowatts.
- `battery_percent`: The average percentage of battery charge at the time of measurement.
- `battery_kwh`: The average amount of power in the battery at the time of measurement in kilowatt hours.
- various weather data, including temperature, humidity, precipitation, wind speed, and solar radiation (ghi, dni, dhi).

In [ ]:
data = pd.read_csv("solar.csv")

In [ ]:
data["timestamp"] = pd.to_datetime(data["timestamp"])

In [ ]:
data["year"] = data["timestamp"].dt.year
data["month"] = data["timestamp"].dt.month_name()
data["day"] = data["timestamp"].dt.day
data["hour"] = data["timestamp"].dt.hour
data["weekday"] = data["timestamp"].dt.day_name()
data["is_weekend"] = np.where(data["weekday"].isin(["Saturday", "Sunday"]), 1, 0)   

In [ ]:
data.set_index("timestamp", inplace=True)

In [ ]:
data.head()

# Animation

Animation is a powerful tool for visualizing time series data. It allows us to see how the data changes over time, making it easier to identify trends and patterns. In this notebook, we will create an animated plot of solar power generation over time.

Let's first start with a static plot of the data.

In [ ]:
data_1week = data["2024-07-01":"2024-07-07"]

plt.figure(figsize=(12, 6))
sns.lineplot(data=data_1week, x=data_1week.index, y="from_pv_kw")
plt.title("Solar Power Generation Over Time")
plt.xlabel("Date")
plt.ylabel("Power Output (kW)")
plt.show()

Now, let's turn this into an animated plot. We will use the `FuncAnimation` function to create the animation. It takes a custom function `animate()` that updates the plot for each frame of the animation, as well as the number of frames and the interval between frames.

To avoid a constant resizing of the plot, we will set the x and y limits of the plot to the minimum and maximum values of the data. This will ensure that the plot does not resize during the animation.

In [ ]:
xmin, xmax = data_1week.index.min(), data_1week.index.max()
ymin, ymax = data_1week["from_pv_kw"].min(), data_1week["from_pv_kw"].max()

fig, ax = plt.subplots(figsize=(12, 6))

def animate(i):
    ax.clear()
    subset = data_1week.iloc[: i + 1]
    sns.lineplot(x=subset.index, y=subset["from_pv_kw"], ax=ax)
    ax.set_title(f"Frame {i}")
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    return ax.collections

anim = FuncAnimation(fig,
                     func=animate,
                     frames=len(data_1week),
                     interval=100,
                     blit=True)

plt.close(fig)
HTML(anim.to_jshtml())
